In [136]:
from torchtune.modules import RotaryPositionalEmbeddings

import torch

rope = RotaryPositionalEmbeddings(4)

x = torch.ones([2, 4, 1, 4])

print(rope(x, input_pos=torch.tensor([[4, 3, 2, 1], [1, 2, 3, 56]])))

tensor([[[[ 0.1032, -1.4104,  0.9592,  1.0392]],

         [[-1.1311, -0.8489,  0.9696,  1.0295]],

         [[-1.3254,  0.4932,  0.9798,  1.0198]],

         [[-0.3012,  1.3818,  0.9900,  1.0099]]],


        [[[-0.3012,  1.3818,  0.9900,  1.0099]],

         [[-1.3254,  0.4932,  0.9798,  1.0198]],

         [[-1.1311, -0.8489,  0.9696,  1.0295]],

         [[ 1.3748,  0.3317,  0.3161,  1.3784]]]])


In [1]:
from foreduce.vampire.vampire import VampireInteractive, VampireAutomatic
import os
from dotenv import load_dotenv
from torch_geometric.utils.convert import from_networkx
from torch_geometric.utils import index_to_mask
import torch

from foreduce.transformer.model import Model
from foreduce.data.data import _type_mapping

load_dotenv()
VAMPIRE = os.getenv("VAMPIRE")

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from foreduce.data.data import GraphDataset

trainset = GraphDataset.load("data/666_train.pt")

/home/apluska/foreduce/foreduce/data/data.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  max_arity, data = torch.load(path)


In [3]:
model = Model(len(_type_mapping), trainset.max_arity, 4, 4, 16, n_heads=4)

In [4]:
from torch_geometric.loader import DataLoader

loader = DataLoader(trainset, batch_size=1)

In [ ]:
result = model(next(iter(loader)))

In [6]:
result

tensor([[-1.7053, -1.6392, -1.6332, -1.6455, -0.6188,  0.1303, -1.6843, -1.6946,
         -1.7016, -1.6941, -1.6994, -1.6870, -1.6832, -1.6899, -1.7007, -1.6959,
         -1.6978, -1.6939, -1.6829, -1.6406, -1.6982, -1.6977, -1.6956, -1.7009,
         -1.6863, -1.6865, -1.6945, -1.7000, -1.6467, -1.7028, -1.6899, -1.6873,
         -1.6918, -1.7000, -1.6948, -1.7016, -1.6974, -1.6871, -1.6886, -1.6995,
         -1.6978, -1.7002, -1.7047, -1.6892, -1.6881, -1.6956, -1.6985, -1.6970,
         -1.7097, -1.6962, -1.6901, -1.6940, -1.6521, -1.6963, -1.7099, -1.7027,
         -1.6914, -1.6902, -1.6994, -1.6982, -1.7079, -1.7105, -1.6951, -1.6907,
         -1.6970, -1.6995, -1.7042, -1.7171, -1.7018, -1.6921, -1.6949, -1.7001,
         -1.7016, -1.7188, -1.7094, -1.6956, -1.6920, -1.6997, -1.7006, -1.7167,
         -1.7175, -1.7003, -1.6910, -1.6984, -1.7006, -1.7124, -1.7240, -1.7063,
         -1.6928, -1.6961, -1.7007, -1.7081, -1.7268, -1.7135, -1.6961, -1.6922,
         -1.7002, -1.7039, -

In [7]:
model.training_step(next(iter(loader)), 0)

ValueError: Target size (torch.Size([213])) must be the same as input size (torch.Size([1, 213]))

In [12]:
automatic = VampireAutomatic(VAMPIRE, f'./problems/SWV/SWV267-2.p')
automatic.run()

symbols = automatic.problem.function_symbols() | automatic.problem.predicate_symbols()
permutation = torch.randperm(32)
name = {s: permutation[i] + 1 for i, s in enumerate(symbols)}

In [13]:
graph, mapping, clauses, labels = automatic.problem.to_graph_data(automatic.tree)

In [14]:
from foreduce.data.data import _type_mapping

data = from_networkx(graph)
data.type = torch.tensor([_type_mapping[t] for t in data.type], dtype=torch.int)
data.name = torch.tensor([name[s] if s is not None else 0 for s in data.name], dtype=torch.int)
data.arity = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.arity], dtype=torch.int)
data.pos = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.pos], dtype=torch.int)
data.clauses = index_to_mask(torch.tensor(clauses), size=data.num_nodes)

In [15]:
model.predict(data)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [93]:
interactive = VampireInteractive(VAMPIRE, f'./problems/GRP/GRP660-12.p')
interactive.__enter__()
symbols = interactive.problem.function_symbols() | interactive.problem.predicate_symbols()
permutation = torch.randperm(32)
name = {s: permutation[i] + 1 for i, s in enumerate(symbols)}

graph = None

In [118]:
if graph is None:
    graph, mapping, clauses = interactive.problem.to_graph(depth=4)
else:
    graph, mapping, clauses = interactive.problem.extend_graph(graph, mapping, len(clauses), depth=4)
data = from_networkx(graph)
data.type = torch.tensor([_type_mapping[t] for t in data.type], dtype=torch.int)
data.name = torch.tensor([name[s] if s is not None else 0 for s in data.name], dtype=torch.int)
data.arity = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.arity], dtype=torch.int)
data.pos = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.pos], dtype=torch.int)
data.clauses = index_to_mask(torch.tensor(clauses), size=data.num_nodes)
score, topk = model.predict(data)
deduped = []
for clause in interactive.problem.clauses:
    if clause not in deduped:
        deduped.append(clause)
_mapping = {i: deduped.index(clause) for i, clause in enumerate(interactive.problem.clauses)}
vals = [(score[topk.index(_mapping[i])].item(), i) for i in range(len(interactive.problem.clauses)) if (not interactive.active[i] and _mapping[i] in topk)]
_, next_clause = max(vals)
interactive.step(next_clause)
interactive.finished

False

In [115]:
interactive

 0: mult(X0, ld(X0, X1)) = X1
 1: ld(X2, mult(X2, X3)) = X3
2: mult(rd(X4, X5), X5) = X4
 3: rd(mult(X6, X7), X7) = X6
 4: mult(mult(mult(X8, X9), X10), X8) = mult(X8, mult(X9, mult(X10, X8)))
 5: goal_0 | x0 = mult(rd(x1, x1), x0)
 6: ~goal_0
 7: mult(mult(mult(X0, X1), X2), mult(X0, mult(X3, mult(mult(X0, X1), X2)))) = mult(mult(mult(X0, mult(X1, mult(X2, X0))), X3), mult(mult(X0, X1), X2))
8: mult(mult(X0, X1), mult(X2, mult(X0, mult(X0, X1)))) = mult(mult(X0, mult(X1, mult(X2, X0))), mult(X0, X1))
9: mult(mult(mult(X0, X3), mult(mult(X0, X1), X2)), mult(X0, mult(X4, mult(mult(X0, X3), mult(mult(X0, X1), X2))))) = mult(mult(mult(X0, mult(X3, mult(X0, mult(X1, mult(X2, X0))))), X4), mult(mult(X0, X3), mult(mult(X0, X1), X2)))
 10: mult(mult(mult(mult(mult(X5, X6), X7), X9), mult(mult(X5, mult(X6, mult(X7, X5))), X8)), mult(mult(mult(X5, X6), X7), mult(X10, mult(mult(mult(mult(X5, X6), X7), X9), mult(mult(X5, mult(X6, mult(X7, X5))), X8))))) = mult(mult(mult(mult(mult(X5, X6), X7), mu

In [120]:
print(interactive.proof)

% Running in auto input_syntax mode. Trying TPTP
[SA] new: 1. mult(X0,ld(X0,X1)) = X1 [input]
[SA] new: 2. ld(X2,mult(X2,X3)) = X3 [input]
[SA] new: 3. mult(rd(X4,X5),X5) = X4 [input]
[SA] new: 4. rd(mult(X6,X7),X7) = X6 [input]
[SA] new: 5. mult(mult(mult(X8,X9),X10),X8) = mult(X8,mult(X9,mult(X10,X8))) [input]
[SA] new: 6. goal_0 | x0 != mult(rd(x1,x1),x0) [input]
[SA] new: 7. ~goal_0 [input]
[SA] new: 8. mult(mult(mult(X0,X1),X2),mult(X0,mult(X3,mult(mult(X0,X1),X2)))) = mult(mult(mult(X0,mult(X1,mult(X2,X0))),X3),mult(mult(X0,X1),X2)) [superposition 5,5]
[SA] new: 9. mult(mult(X0,X1),mult(X2,mult(X0,mult(X0,X1)))) = mult(mult(X0,mult(X1,mult(X2,X0))),mult(X0,X1)) [superposition 5,5]
[SA] new: 10. mult(mult(mult(X0,X3),mult(mult(X0,X1),X2)),mult(X0,mult(X4,mult(mult(X0,X3),mult(mult(X0,X1),X2))))) = mult(mult(mult(X0,mult(X3,mult(X0,mult(X1,mult(X2,X0))))),X4),mult(mult(X0,X3),mult(mult(X0,X1),X2))) [superposition 8,5]
[SA] new: 11. mult(mult(mult(mult(mult(X5,X6),X7),X9),mult(mult(

In [48]:
data = from_networkx(graph)
data.type = torch.tensor([_type_mapping[t] for t in data.type], dtype=torch.int)
data.arity = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.arity], dtype=torch.int)
data.pos = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.pos], dtype=torch.int)
data.clauses = index_to_mask(torch.tensor(clauses), size=data.num_nodes)
score, topk = model.predict(data)
deduped = []
for clause in interactive.problem.clauses:
    if clause not in deduped:
        deduped.append(clause)
_mapping = {i: deduped.index(clause) for i, clause in enumerate(interactive.problem.clauses)}
vals = [(score[topk.index(_mapping[i])].item(), i) for i in range(len(interactive.problem.clauses)) if (not interactive.active[i] and _mapping[i] in topk)]
_, next_clause = max(vals)
interactive.step(next_clause)
graph, mapping, clauses = interactive.problem.extend_graph(graph, mapping, len(clauses), depth=4)
interactive.finished

False

In [49]:
interactive

0: product(identity, X0, X0)
 1: product(inverse(X1), X1, identity)
2: product(X2, X3, multiply(X2, X3))
3: equalish(X6, X7) | ~product(X4, X5, X7) | ~product(X4, X5, X6)
 4: product(X8, X12, X13) | ~product(X10, X11, X13) | ~product(X9, X11, X12) | ~product(X8, X9, X10)
 5: product(X16, X17, X19) | ~product(X14, X18, X19) | ~product(X15, X17, X18) | ~product(X14, X15, X16)
6: product(X22, X23, X21) | ~product(X22, X23, X20) | ~equalish(X20, X21)
 7: product(X0, X1, identity) | ~product(X2, X3, X1) | ~product(X0, X2, inverse(X3))
 8: ~product(X4, identity, X5) | product(X6, X7, X5) | ~product(X4, inverse(X7), X6)
 9: product(identity, X0, X1) | ~product(inverse(inverse(X0)), identity, X1)